In [102]:
%load_ext autoreload
%autoreload 2

import importlib
try:    importlib.reload(Jupytils)
except: import Jupytils
import logging as log
import itertools as it
showTopbar('''ARX Model Create''')

from sklearn.linear_model import LinearRegression
from arch.univariate import ARX

In [103]:
df=LoadDataSet("data/test.csv")
dfi=LoadDataSet("data/test.xml", xmlTag="Invariant")
dfi['theta1']= dfi.apply (lambda row: [ float(i.strip()) for i in  row['theta'].split(',') if i.strip()], axis=1)
dfi.n = dfi.n.astype(int)
dfi.m = dfi.m.astype(int)
dfi.k = dfi.k.astype(int)
dfi.fitness = dfi.fitness.astype(float)
dfi.threshold = dfi.threshold.astype(float)
df

,Time,A,B,C,D,E,F,G
0,60000,94.502,156.473,148.500,64.516,25.060,50.310,47.876
1,120000,94.806,156.706,148.500,64.115,24.957,52.319,47.934
2,180000,94.602,156.691,148.500,64.189,24.691,53.450,48.524
3,240000,94.670,156.496,148.500,62.643,24.726,53.904,49.280
...,...,...,...,...,...,...,...,...
2497,149880000,94.679,157.041,148.500,61.631,25.016,50.494,48.733
2498,149940000,94.594,156.820,148.500,61.752,25.122,51.641,48.855
2499,150000000,95.229,156.739,148.500,62.567,25.575,52.215,49.056
2500,150060000,94.884,156.338,148.500,63.870,25.444,53.483,49.159


In [104]:
#display(HTML(dfi.to_html()))
dfi.sort_values(['uName', 'yName'], inplace=True)
dfi = dfi.reset_index(drop=True)
displayDFs(dfi, showIcons=False);

,uName,yName,fitness,correlation,validity,theta,n,m,k,threshold,slope,intercept,theta1
0,A,B,0.836,1.0,1.0,"0.7566798046980097,0.08742089615341801,76.55782422422585",0,1,0,4.618,0.0,0.0,"[0.7566798046980097, 0.08742089615341801, 76.55782422422585]"
1,A,D,0.297,1.0,1.0,"-0.9630133424708321,0.023043866594117834,0.10041268310120169",1,0,2,3.407,0.0,0.0,"[-0.9630133424708321, 0.023043866594117834, 0.10041268310120169]"
2,A,E,0.162,1.0,1.0,"0.34367147588451813,-0.34042215819690835,24.711512869990965",0,1,0,2.284,0.0,0.0,"[0.34367147588451813, -0.34042215819690835, 24.711512869990965]"
3,A,F,0.721,1.0,1.0,"-0.7120381334303367,0.20514306199581256,0.18059555568220276,-21.856425676200672",1,1,2,5.690,0.0,0.0,"[-0.7120381334303367, 0.20514306199581256, 0.18059555568220276, -21.856425676200672]"
4,A,G,0.125,1.0,1.0,"-0.5345073561536757,-0.0029261322711296245,-0.044098931439243885,27.32336917086615",1,1,0,3.285,0.0,0.0,"[-0.5345073561536757, -0.0029261322711296245, -0.044098931439243885, 27.32336917086615]"
5,B,A,0.843,1.0,1.0,"0.5722391154919677,0.5838044266562292,-86.3624790442679",0,1,0,4.155,0.0,0.0,"[0.5722391154919677, 0.5838044266562292, -86.3624790442679]"


In [105]:
from invx import *

In [106]:
%time dfii = CreateInvariants("data/test.csv", '/tmp/test/test.model')
displayDFs(dfii);

INFO:root:Creating invariants using Index(['A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='object')


CPU times: user 17.2 s, sys: 544 ms, total: 17.7 s
Wall time: 15.9 s


,uName,yName,fitness,correlation,theta,n,m,k,threshold
0,A,B,0.836,0.986,"0.7566798046979863,0.08742089615344534,76.55782422422541",0,1,0,4.618
1,A,D,0.297,0.628,"0.963013342470833,0.02304386659411589,0.10041268310138918",1,0,2,3.407
2,A,E,0.162,0.078,"0.3436714758845351,-0.34042215819692534,24.711512869990973",0,1,0,2.284
3,A,F,0.721,0.942,"0.7120381334303055,0.20514306199551868,0.18059555568252303,-21.856425676201695",1,1,2,5.690
4,A,G,0.125,-0.481,"0.534507356153688,-0.002926132271156909,-0.04409893143921596,27.323369170865533",1,1,0,3.285
5,B,A,0.843,0.986,"0.5722391154920231,0.583804426656154,-86.36247904426479",0,1,0,4.155


In [ ]:
%%time
for i in range(len(dfi)):
    loc=i
    u,v,f,c,_,t,n,m,k,thr,_,_,etheta=dfi.loc[loc]
    x=df[u].values    
    y=df[v].values
    best=findBest(y,x)
    ret = best
    inv1 = [u,v,ret.fitscore[0], 1, ret.res, ret.nmk[0], ret.nmk[1], ret.nmk[2], ret.threshold]
    print(u,v,ret.fitscore[0], ",".join([str(c) for c in ret.res]), ret.nmk[0], ret.nmk[1], ret.nmk[2], ret.threshold)
    print(u,v,f,t,n,m,k,thr,_,_,etheta)
    if ( not np.allclose(f, best.fitscore) or (n,m,k) != best.nmk  or not np.allclose(thr, best.threshold)):
        print(f'{u,v} Fit: {f} === {best.fitscore}, \nnmk: {(n,m,k)} === {best.nmk}, \n Threshhold: {thr} === {best.threshold}')
        
    break;
#def testCreate()
#CreateInvariants("data/test.csv")

In [ ]:
def testARXModelLR(dfi, loc=0, log=True):
    u,v,f,c,_,t,n,m,k,_,_,_,theta=dfi.loc[loc]

    x=df[u].values    
    y=df[v].values
    x1=x.reshape((len(x),1))
    logd(log,f'{loc}: {u,v} n,m,k ={n,m,k} theta={ np.array(theta)}' )
    #res, arx = ARXModelOld(y, x1, n, m, k);
    ret, arx, ret1 = ARXModelLR(y, x1, n, m, k);
    np.set_printoptions(precision=6)
    logd(log, ret, ret1)
    
    comp = np.allclose(ret1,theta)
    logd(log, "Matched: ", comp)
    if ( not comp):
        print (f"{loc} {n,m,k} {u, v } Not Matched\n {ret1} \n {theta}")
    return ret, comp, ret1, theta;


for i in range(30):
    ret, comp,ret1, etheta = testARXModelLR(dfi, i, False)
    
print("If there was mot match you would see a print out! Nothing printed means all is well")


In [ ]:
def testFitnessScore(dfi, loc = 0, log=True):
#loc=9
    u,v,f,c,_,t,n,m,k,_,_,_,theta=dfi.loc[loc]

    x=df[u].values    
    y=df[v].values
    x1=x.reshape((len(x),1))
    #logd(log,f'{loc}: {u,v} n,m,k ={n,m,k} theta={ np.array(theta)}' )
    theta1, arx,_ = ARXModelLR(y,x1,n,m,k)
    fitscore,yh,rs = FitnessScore(x,y,n,m,k, theta1, True)

    ret = np.allclose(fitscore, f)
    if (not ret):
        print(f"=> {loc} Score No Matched {f}: {n,m,k} \n {f} {theta}\n {fitscore}{theta1}")
        #return (ret)

for i in range(len(dfi)):
    testFitnessScore(dfi, i, False)
  

In [85]:
u1,v1,f1,c1,_,t1,n1,m1,k1,_,_,_,theta1 = dfi1.loc[2]
u1

'D'

In [113]:
def compareDFI(dfi1, dfi2, loc =0, log=True):
    u1,v1,f1,c1,_,t1,n1,m1,k1,_,_,_,theta1=dfi1.loc[loc]
    
    r = dfi2[(dfi2.uName == u1) & ( dfi2.yName == v1)]
    #u,v,f,c,_,t,n,m,k,thr,theta=dfi2.loc[loc]
    u2,v2,f2,c2,t2,n2,m2,k2,th2,theta2=r.values[0]
    theta2 = np.array(theta2)
    theta2[0:n2] *= -1;
    
    ret = True
    if ( not np.allclose(f1, f2) ):
        print(f"Location: {loc} => Fitness score not match {f1} != {f2}")
        ret = False
        
    if (n1 != n2 or m1 != m2 or k1 != k2):
        print(f"Location: {loc} => nmk not match {n1, m1, k1} != {n2, m2, k2}")
        ret = False

    if ( not np.allclose(theta1, theta2) ):
        print(f"Location: {loc} => Parameters didnt match {theta1} != {theta2}")
        ret = False

    if ( not ret):
        print (len(r), u1, v1, r)
    else:
        print(loc, ' ', end='')    
    return ret;

def testCompareInvCSV(invFile, invCSV, log=True):
    dfi1=LoadDataSet( invFile, xmlTag="Invariant")
    dfi1['theta1']= dfi1.apply (lambda row: [ float(i.strip()) for i in  row['theta'].split(',') if i.strip()], axis=1)
    dfi1.n = dfi1.n.astype(int)
    dfi1.m = dfi1.m.astype(int)
    dfi1.k = dfi1.k.astype(int)
    dfi1.fitness = dfi1.fitness.astype(float)
    dfi1.threshold = dfi1.threshold.astype(float)


    dfi2 = pd.read_csv(invCSV)
    dfi2['theta1']= dfi2.apply (lambda row: [ float(i.strip()) for i in  row['theta'].split(',') if i.strip()], axis=1)
    dfi2.n = dfi2.n.astype(int)
    dfi2.m = dfi2.m.astype(int)
    dfi2.k = dfi2.k.astype(int)
    dfi2.fitness = dfi2.fitness.astype(float)
    dfi2.threshold = dfi2.threshold.astype(float)

    for i in range(len(dfi1)):
        ret = compareDFI(dfi1, dfi2, i, True)
        if ( not ret):
            break
        
    return dfi1, dfi2

dfi1, dfi2 = testCompareInvCSV ("data/test.xml",'/tmp/test/test.model')


0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  

In [9]:
dfi1

,uName,yName,fitness,correlation,validity,theta,n,m,k,threshold,slope,intercept,theta1
0,A,D,0.297,1.0,1.0,"-0.9630133424708321,0.023043866594117834,0.10041268310120169",1,0,2,3.407,0.0,0.0,"[-0.9630133424708321, 0.023043866594117834, 0.10041268310120169]"
1,A,G,0.125,1.0,1.0,"-0.5345073561536757,-0.0029261322711296245,-0.044098931439243885,27.32336917086615",1,1,0,3.285,0.0,0.0,"[-0.5345073561536757, -0.0029261322711296245, -0.044098931439243885, 27.32336917086615]"
2,D,A,0.221,1.0,1.0,"0.7481099846553998,45.251285815522856",0,0,0,16.191,0.0,0.0,"[0.7481099846553998, 45.251285815522856]"
3,G,A,0.151,1.0,1.0,"-1.4284761171766727,-1.3817957176695685,228.37461123903475",0,1,0,13.373,0.0,0.0,"[-1.4284761171766727, -1.3817957176695685, 228.37461123903475]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,E,G,0.006,1.0,1.0,"-0.6435966390862123,-0.1143641835401605,-0.15006304176428062,24.34198722193917",1,1,0,3.163,0.0,0.0,"[-0.6435966390862123, -0.1143641835401605, -0.15006304176428062, 24.34198722193917]"
27,G,E,0.002,1.0,1.0,"-0.02273742369863696,0.024592471721276675,24.907059054326485",0,1,0,1.327,0.0,0.0,"[-0.02273742369863696, 0.024592471721276675, 24.907059054326485]"
28,F,G,0.285,1.0,1.0,"-0.4189162868732759,0.36024024243855673,52.25831775755052",0,1,0,2.884,0.0,0.0,"[-0.4189162868732759, 0.36024024243855673, 52.25831775755052]"
29,G,F,0.301,1.0,1.0,"-0.9680940617351281,-1.583506978250862,1.190675977845462,20.907965553900514",1,1,1,5.990,0.0,0.0,"[-0.9680940617351281, -1.583506978250862, 1.190675977845462, 20.907965553900514]"


In [99]:
displayDFs([dfi1, dfi2]);